In [26]:
# clear all the variables
%reset -f
import numpy as np
import pandas as pd
import sys
import turicreate as tc
import os
from os import walk
import scipy.io
import xlrd, os, fnmatch
import pandas as pd
import tensorflow as tf
import plotly.plotly as py
import plotly.graph_objs as go
from scipy.spatial.distance import cdist
# Path variable (where the data is located)
myPath = './data/Expt_I/' 
# Append the path variable to existing search path
sys.path.append(myPath)
# Get the file information in the directory
file_list = []
ignore_list = ['pilot']
for root, dirs, files in os.walk(myPath):
    if 'pilot' in dirs:
        dirs[:]=[]
    for filename in files:
        if fnmatch.fnmatch(filename.lower(), "*csv"):
            file_list.append(os.path.join(root, filename))

In [27]:
file_list

['./data/Expt_I/4_12Nov2018_20-00.csv',
 './data/Expt_I/10_14Nov2018_15-29.csv',
 './data/Expt_I/5_13Nov2018_15-08.csv',
 './data/Expt_I/9_14Nov2018_14-35.csv',
 './data/Expt_I/15_15Nov2018_16-54.csv',
 './data/Expt_I/6_13Nov2018_16-14.csv',
 './data/Expt_I/2_12Nov2018_17-35.csv',
 './data/Expt_I/13_15Nov2018_15-04.csv',
 './data/Expt_I/12_15Nov2018_13-41.csv',
 './data/Expt_I/3_12Nov2018_18-35.csv',
 './data/Expt_I/7_13Nov2018_17-00.csv',
 './data/Expt_I/8_14Nov2018_13-31.csv',
 './data/Expt_I/11_15Nov2018_11-36.csv',
 './data/Expt_I/14_15Nov2018_16-06.csv']

In [28]:
data = tc.SFrame.read_csv(file_list[0])
temp = data
for i in range(1,len(file_list)):
    temp = tc.SFrame.read_csv(file_list[i])
    data = data.append(temp)
len(data)

Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/Expt_I/4_12Nov2018_20-00.csv

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,str,float,int,str,str,float,float,array,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Parsing completed. Parsed 100 lines in 0.04553 secs.

Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/Expt_I/4_12Nov2018_20-00.csv

Parsing completed. Parsed 420 lines in 0.015856 secs.

...

Parsing completed. Parsed 100 lines in 0.009823 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,str,float,int,str,str,float,float,array,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/Expt_I/14_15Nov2018_16-06.csv

Parsing completed. Parsed 420 lines in 0.010849 secs.

5880

In [29]:
from plotly import tools
import colorlover as cl
trace = [[]]*2
colors = cl.scales['3']['seq']['YlOrRd']
for i,cuePos in enumerate(data['cueSpatialPos1'].unique()):
    cuePosdata = data[data['cueSpatialPos1']==cuePos]
    temp1 = []
    for j,ecc in enumerate(cuePosdata['cueEccentricity1'].unique().sort()):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==ecc]
        temp1.append(go.Histogram(x = ecc_data['responsePosRelative0'].to_numpy(), 
                                  histnorm='probability',name = str(ecc) + 'deg', opacity = 1, marker = dict(color = colors[j])))
    trace[i] = temp1

if data['cueSpatialPos1'].unique()[0] == [-1.0,0.0]:
    titles = ('West','East')
else:
    titles = ('East','West')
    
fig = tools.make_subplots(rows=1, cols=2, subplot_titles = titles)
fig['layout'].update(barmode='group', title = 'performance in RSVP visual now task for different cue positions - '+str(len(data)/420)+' participants - ISI (73 msec)')
#fig['layout']['xaxis3'].update(title='position of reported item relative to the true item', range = [-2,2])
#fig['layout']['xaxis4'].update(title='position of reported item relative to the true item', range = [-2,2])
fig['layout']['xaxis1'].update(range=[-3,3],title='position of reported item relative to the true item')
fig['layout']['xaxis2'].update(range=[-3,3],title='position of reported item relative to the true item')
fig['layout']['yaxis1'].update(title='Probability')
#fig['layout']['yaxis3'].update(title='Frequency')
for i in range(3):
    fig.add_trace(trace[0][i], 1, 1)
    fig.add_trace(trace[1][i], 1, 2)
    #fig.append_trace(trace[2][i], 2, 1)
    #fig.append_trace(trace[3][i], 2, 2)
py.iplot(fig, filename='RSVP_visual_now')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [31]:
trace_data = []
for j,ecc in enumerate(cuePosdata['cueEccentricity1'].unique().sort(ascending=False)):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==ecc]
        trace_data.append(go.Histogram(x = ecc_data['responsePosRelative0'].to_numpy(), 
                                  histnorm='probability',name = str(ecc) + 'deg', opacity = 1, marker = dict(color = colors[j])))

        
fig = tools.make_subplots(rows=1, cols=1)
fig['layout'].update(barmode='group', title = 'performance in RSVP visual now task for different cue positions - '+str(len(data)/420)+' participants - ISI (73 msec)')
fig['layout']['xaxis1'].update(range = [-3,3], title='position of reported item relative to the true item')
fig['layout']['yaxis1'].update(title='Probability', range = [0,0.8])
for i in range(3):
    fig.add_trace(trace_data[i], 1, 1)
py.iplot(fig, filename='RSVP_visual_now_grouped')

This is the format of your plot grid:
[ (1,1) x1,y1 ]



/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [7]:
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

In [8]:
X_plot = np.linspace(-3, 3, 1000)
trace_data = []
distributions = []
fill_colors = ['rgba(255,237,160,0.2)', 'rgba(254,178,76,0.2)', 'rgba(240,59,32,0.2)']
for j,ecc in enumerate(cuePosdata['cueEccentricity1'].unique().sort()):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==ecc]
        X = ecc_data['responsePosRelative0'].to_numpy()
        # Gaussian KDE
        #kde = KernelDensity(kernel='gaussian').fit(X)
        kde = gaussian_kde(X)
        log_dens = kde(X_plot)
        distributions.append({'ecc' : ecc, 'dist' : X, 'dens': log_dens})
        trace_data.append(go.Scatter(x = X_plot, y = log_dens,
                                  mode='lines', fill = 'tozeroy', fillcolor = fill_colors[j], name = str(ecc) + 'deg', line = dict(color = colors[j], width = 2) ))
        
fig = tools.make_subplots(rows=1, cols=1)
fig['layout'].update(barmode='group', title = 'performance in RSVP visual now task for different cue positions - '+str(len(data)/420)+' participants - ISI (73 msec)')
fig['layout']['xaxis1'].update(title='position of reported item relative to the true item')
fig['layout']['yaxis1'].update(title='Probability', range = [0,0.8])
for i in range(2):
    fig.add_trace(trace_data[i], 1, 1)
py.iplot(fig, filename='RSVP_visual_now_grouped')


This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [9]:
import numpy as np
from scipy.stats import kurtosis, skew

for x in distributions:
    print( 'Eccentricity: {}'.format( x['ecc']))
    print( 'excess kurtosis of normal distribution (should be 0): {}'.format( kurtosis(x['dens']) ))
    print( 'skewness of normal distribution (should be 0): {}'.format( skew(x['dens']) ))
    print( '\n')

Eccentricity: 2
excess kurtosis of normal distribution (should be 0): -1.2243302210938152
skewness of normal distribution (should be 0): 0.4831588522978982


Eccentricity: 6
excess kurtosis of normal distribution (should be 0): -1.4099260895813546
skewness of normal distribution (should be 0): 0.052620480836299416




In [10]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

hsb = data.to_dataframe()
mod = ols('responsePosRelative0 ~ C(cueEccentricity1)',
                data=hsb).fit()
                
aov_table = sm.stats.anova_lm(mod, typ=2)
print(aov_table)

                           sum_sq      df         F    PR(>F)
C(cueEccentricity1)      2.288435     1.0  0.232425  0.629749
Residual             57874.168027  5878.0       NaN       NaN


In [11]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[1]['ecc']))
scipy.stats.ttest_ind(distributions[0]['dist'], distributions[1]['dist'],axis =0, equal_var=True)

Ecc:2 vs.Ecc:6


Ttest_indResult(statistic=0.20847694714909965, pvalue=0.8348710319857455)

In [12]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
scipy.stats.ttest_ind(distributions[0]['dist'], distributions[2]['dist'], axis =0,equal_var=True)

IndexError: list index out of range

In [13]:
print('Ecc:{}'.format(distributions[1]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
scipy.stats.ttest_ind(distributions[1]['dist'], distributions[2]['dist'], axis =0,equal_var=True)

Ecc:6 vs.Ecc:10


Ttest_indResult(statistic=-1.0189619194236839, pvalue=0.30834693037006633)

In [13]:
import turicreate.aggregate as agg
sub_data = data.groupby(['subject','cueEccentricity1'],operations = {'avg_lag':agg.AVG('responsePosRelative0')})
ratios = []
for sub in data['subject'].unique().sort():
    temp1 = data[data['subject']==sub]
    for ecc in temp1['cueEccentricity1'].unique().sort():
        temp = temp1[temp1['cueEccentricity1']==ecc]
        count_data = temp['responsePosRelative0'].value_counts()
        ratios.extend(count_data[count_data['value']==-1]['count']-count_data[count_data['value']==1]['count'])
sub_data = sub_data.sort(['subject','cueEccentricity1'])
skewness = []
kurtosissy = []
for sub in data['subject'].unique().sort():
    subd = data[data['subject']==sub]
    for ecc in subd['cueEccentricity1'].unique().sort():
        eccd = subd[subd['cueEccentricity1']==ecc]
        X = eccd['responsePosRelative0'].to_numpy()
        kde = gaussian_kde(X)
        log_dens = kde(X_plot)
        skewness.append(skew(log_dens))
        kurtosissy.append(kurtosis(log_dens))

sub_data['skew'] = skewness
sub_data['kurtosis'] = kurtosissy
sub_data['ratio'] = ratios

In [14]:
sub_hsb = sub_data.to_dataframe()
#sub_hsb = hsb.groupby(['subject','cueEccentricity1']).apply(skew('responseRelative0'))
mod = ols('skew ~ C(cueEccentricity1)',
                data=sub_hsb).fit()
                
aov_table = sm.stats.anova_lm(mod)
aov_table

,df,sum_sq,mean_sq,F,PR(>F)
C(cueEccentricity1),1.0,1.811945,1.811945,5.725689,0.021766
Residual,38.0,12.025436,0.316459,NaN,NaN


In [15]:
mod1 = ols('kurtosis ~ C(cueEccentricity1)',
                data=sub_hsb).fit()
                
aov_table1 = sm.stats.anova_lm(mod1)
aov_table1

,df,sum_sq,mean_sq,F,PR(>F)
C(cueEccentricity1),1.0,1.756532,1.756532,2.187487,0.147379
Residual,38.0,30.513651,0.802991,NaN,NaN


In [16]:
deg2_data = sub_data[sub_data['cueEccentricity1']==2]
deg2_data = deg2_data.sort('subject')
deg6_data = sub_data[sub_data['cueEccentricity1']==6]
deg6_data = deg6_data.sort('subject')
#deg10_data = sub_data[sub_data['cueEccentricity1']==10]
#deg10_data = deg10_data.sort('subject')

deg2 = deg2_data['avg_lag'].to_numpy()
deg2_ratio = deg2_data['ratio'].to_numpy()
deg6 = deg6_data['avg_lag'].to_numpy()
deg6_ratio = deg6_data['ratio'].to_numpy()
#deg10 = deg10_data['avg_lag'].to_numpy()
#deg10_ratio = deg10_data['ratio'].to_numpy()

In [20]:
len(deg2_ratio)

20

In [17]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs. zero mean')
scipy.stats.ttest_1samp(deg2,0.0)

Ecc:2 vs. zero mean


Ttest_1sampResult(statistic=2.3740959323908175, pvalue=0.028286059250607242)

In [18]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[1]['ecc']))
scipy.stats.ttest_rel(deg2, deg6)

Ecc:2 vs.Ecc:6


Ttest_relResult(statistic=-0.3961806965398601, pvalue=0.6963857146041255)

In [21]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
scipy.stats.ttest_rel(deg2,deg10)

Ecc:2 vs.Ecc:10


Ttest_relResult(statistic=1.9448065601268494, pvalue=0.07375444777092569)

In [22]:
print('Ecc:{}'.format(distributions[1]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
scipy.stats.ttest_rel(deg6,deg10)

Ecc:6 vs.Ecc:10


Ttest_relResult(statistic=-1.0574620602273954, pvalue=0.30955571320934083)

In [21]:
output = tc.SFrame()
output['subject'] = np.arange(1,21)
output['avg_lag_2deg'] = deg2
output['avg_lag_6deg'] = deg6
#output['avg_lag_10deg'] = deg10

output['ratio_2deg'] = deg2_ratio
output['ratio_6deg'] = deg6_ratio
#output['ratio_10deg'] = deg10_ratio

In [22]:
output

subject,avg_lag_2deg,avg_lag_6deg,ratio_2deg,ratio_6deg
1,0.6642857142857138,0.39999999999999997,-22,-8
2,-0.5857142857142856,-0.5357142857142857,-8,-6
3,0.22142857142857136,0.1857142857142858,-26,-30
4,-0.007142857142857155,0.08571428571428572,-8,1
5,0.11428571428571424,-0.07142857142857144,-1,-5
6,0.3285714285714285,-0.0499999999999999,-17,-31
7,0.12142857142857154,0.33571428571428574,-19,-20
8,0.08571428571428567,0.6714285714285714,-16,-29
9,1.4428571428571433,0.9642857142857143,-15,-22
10,0.43571428571428555,0.5428571428571435,-26,-11


In [23]:
print('Ecc:{}'.format(2)+' vs.'+ 'Ecc:{}'.format(6))
scipy.stats.ttest_rel(output['avg_lag_2deg'].to_numpy(), output['avg_lag_6deg'].to_numpy())

Ecc:2 vs.Ecc:6


Ttest_relResult(statistic=-0.3961806965398601, pvalue=0.6963857146041255)

In [26]:
print('Ecc:{}'.format(2)+' vs.'+ 'Ecc:{}'.format(10))
scipy.stats.ttest_rel(output['avg_lag_2deg'].to_numpy(), output['avg_lag_10deg'].to_numpy())

Ecc:2 vs.Ecc:10


Ttest_relResult(statistic=1.9448065601268494, pvalue=0.07375444777092569)

In [27]:
print('Ecc:{}'.format(6)+' vs.'+ 'Ecc:{}'.format(10))
scipy.stats.ttest_rel(output['avg_lag_6deg'].to_numpy(), output['avg_lag_10deg'].to_numpy())

Ecc:6 vs.Ecc:10


Ttest_relResult(statistic=-1.0574620602273954, pvalue=0.30955571320934083)

In [24]:
print('Ecc:{}'.format(2)+' vs.'+ 'Ecc:{}'.format(6))
scipy.stats.ttest_rel(output['ratio_2deg'].to_numpy(), output['ratio_6deg'].to_numpy())

Ecc:2 vs.Ecc:6


Ttest_relResult(statistic=-1.74452331960508, pvalue=0.09722330515174074)

In [29]:
print('Ecc:{}'.format(2)+' vs.'+ 'Ecc:{}'.format(10))
scipy.stats.ttest_rel(output['ratio_2deg'].to_numpy(), output['ratio_10deg'].to_numpy())

Ecc:2 vs.Ecc:10


Ttest_relResult(statistic=-4.290938887495348, pvalue=0.0008778918171453133)

In [30]:
print('Ecc:{}'.format(6)+' vs.'+ 'Ecc:{}'.format(10))
scipy.stats.ttest_rel(output['ratio_6deg'].to_numpy(), output['ratio_10deg'].to_numpy())

Ecc:6 vs.Ecc:10


Ttest_relResult(statistic=0.18887108137108505, pvalue=0.8531105490188067)

In [ ]:
raw_output = tc.SFrame()
raw_output['subject'] = np.repeat(np.arange(1,15),140)
raw_output['lag_2deg'] = raw_deg2
raw_output['lag_6deg'] = raw_deg6
raw_output['lag_10deg'] = raw_deg10

In [ ]:
raw_output.explore()

In [ ]:
#raw_output.export_csv('Expt_1_raw_data.csv')

In [37]:
output['avg_lag_2deg'].std()/np.sqrt(14)

0.06975989748727841

In [41]:
output.explore()

Materializing SFrame

In [25]:
trace_data = []
trace_data.append(go.Bar(x = ['preference to lag -1 over lag 1 responses'],
                        y = [output['ratio_2deg'].mean()],
                        name='2 deg',
                        marker = dict(color = colors[0]),
                        error_y = dict(type = 'data',
                                      array = [output['ratio_2deg'].std()/np.sqrt(14)],
                                      visible=True
                                      )
                        )
                 )

trace_data.append(go.Bar(x = ['preference to lag -1 over lag 1 responses'],
                        y = [output['ratio_6deg'].mean()],
                        name='6 deg',
                        marker = dict(color = colors[1]),
                        error_y = dict(type = 'data',
                                      array = [output['ratio_6deg'].std()/np.sqrt(14)],
                                      visible=True
                                      )
                        )
                 )

trace_data.append(go.Bar(x = ['preference to lag -1 over lag 1 responses'],
                        y = [output['ratio_10deg'].mean()],
                        name='10 deg',
                        marker = dict(color = colors[2]),
                        error_y = dict(type = 'data',
                                      array = [output['ratio_10deg'].std()/np.sqrt(14)],
                                      visible=True
                                      )
                        )
                 )

layout = go.Layout(
    barmode='group',
    title = '14 participants',
    yaxis = dict(title = '# of responses lag(-1) - lag(1)')
)
fig = go.Figure(data=trace_data, layout=layout)
py.iplot(fig, filename='error-bar-bar')

/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [42]:
output['ratio_2deg'].mean()

6.714285714285715

In [43]:
output['ratio_6deg'].mean()

20.71428571428571

In [44]:
output['ratio_10deg'].mean()

20.21428571428571

In [45]:
[output['ratio_2deg'].std()/np.sqrt(14), output['ratio_6deg'].std()/np.sqrt(14), output['ratio_10deg'].std()/np.sqrt(14)]

[3.3249652007210293, 4.887957475253174, 5.63804367162482]